In [ ]:
# Import packages
import TS_Custom
from TS_Custom import *

    

#import FIXEDSTrategy_Multithread_YFData
#from FIXEDSTrategy_Multithread_YFData import *

sellresults_filename = "zlogs_alphaedge_sellresults_df" + ".csv"

    
def getStrategyResult(position):
    #for symbol in symbol_list :
    
    try:
        
        
        #print (datetime.now().strftime('%H:%M'))

        
        # get options data from td ameritrade
        #content = getOptionChainDatafromTD(symbol)

        # retrieve call symbol
        #call_optionSymbol = getCallOptionSymbol(content)

        # retrieve put symbol
        #put_optionSymbol = getPutOptionSymbol(content)   

                 
        #print to console
        #printToConsole(df)
        

        # Strategy Code
        
        symbol = position['SYMBOL']
        positionType = position['POSITION_TYPE']
        quantity =position['QUANTITY']
        avgPrice = position['AVGPRICE']
        bidPrice = position['BID']
        askPrice = position['ASK']
        openpl = position['OPENPL']
        openpl_percent = position['OPENPL_PERCENT']
        triggerprice = position['TRIGGER_PRICE']
        previous_openpl = position['PREVIOUS_OPENPL']
        profit_target = position['PROFIT_TARGET']
        stop_loss = position['STOP_LOSS']
        #limitprice = round((askPrice + bidPrice)/2,2)
        limitprice = round((bidPrice),2)
        
        val = symbol.split(" ")
        
        # get price history from tradestation
        barsBack = 75
        minutes = 2
        if (len(val) > 1):
            df = getPriceHistory_TS(val[0],str(barsBack), str(minutes))
        else :
            df = getPriceHistory_TS(symbol,str(barsBack), str(minutes))
                   
        #if (df != []):
        
        
        
        
        #convert current candles to heiken-ashi
        df = getHA(df)
        #print(df.iloc[[98]])        

        ema9 = df['ema9']        
        
        df = Supertrend(df, 10, 1)
        
        #retrieve keys
        last_key = ema9.keys()[-1]
        previous_key = ema9.keys()[-2]
        previous_key2 = ema9.keys()[-3]
        previous_key3 = ema9.keys()[-4]
        

        #variables
        datetime_now = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        strategy = 'SELL_Supertrend'

        selloptions = False
        
        # if the position is an option then sell according to the rules below
        if (len(val) > 1) :
            if (selloptions):                    

                if (val[1].__contains__("C")):
                    #if (openpl_percent > 2) or (openpl_percent < -8) :
                    if ((df['HA_Close'][previous_key] < df['HA_Open'][previous_key]) and (df['HA_Open'][previous_key] == df['HA_High'][previous_key])) :
                        print (datetime.now().strftime('%H:%M:%S'))
                        print('Sell Call for ', symbol, 'at', limitprice)
                        profitloss = round((quantity * 100 * (limitprice-avgPrice)),2)

                        # Sell Call in TS
                        #sellCALL(symbol, quantity, df,val, tradelog_filename, content)    


                        #add to DB
                        addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H-%M-%S'), str(symbol), 'SELL_CALL', limitprice, quantity)
                        addprofitlossOutputtoDB(profitloss)
                        deletePositionfromDB(symbol)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELL_CALL', limitprice, quantity, profitloss, bidPrice, askPrice,df)



                if (val[1].__contains__("P")):

                    #if (openpl_percent > 2) or (openpl_percent < -8) :
                    if ((df['HA_Close'][previous_key] > df['HA_Open'][previous_key]) and (df['HA_Open'][previous_key] == df['HA_Low'][previous_key])) :
                        print (datetime.now().strftime('%H:%M:%S'))
                        print('Sell Put for ', symbol, 'at', limitprice)
                        profitloss = round((quantity * 100 * (limitprice-avgPrice)),2)

                        #Sell Put in TS
                        #sellPUT(symbol, quantity, df, val, tradelog_filename, content)    


                        #add to DB
                        addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H-%M-%S'), str(symbol), 'SELL_PUT', limitprice, quantity)
                        addprofitlossOutputtoDB(profitloss)
                        deletePositionfromDB(symbol)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELL_PUT', limitprice, quantity, profitloss, bidPrice, askPrice,df)


        # if the position is a stock then sell according to the rules below
        else :

            if (positionType == 'Long'):

                 #if (openpl_percent >= 0.02) or (openpl_percent < -0.60) :                        
                #if (df['ST_Upperband'][last_key] > 0):

                 #if ((openpl > triggerprice) and (df['HA_Close'][previous_key] < df['HA_Open'][previous_key])) or \
                    #(bidPrice <= stop_loss):

                #if (openpl_percent >= 1.00) or (bidPrice <= stop_loss) : 
                
                #if  (df['ST_Upperband'][previous_key]> 0) or (df['ST_Upperband'][last_key]> 0) :
                
                #if (openpl >= triggerprice) or (bidPrice <= stop_loss) :
                
                if (df['ST_Upperband'][previous_key]> 0) and (df['ST_Lowerband'][previous_key2]> 0) :

                #if (openpl_percent >= 0.03 or openpl_percent <= -0.30) :
                #if (openpl_percent >= 0.1) :

                    print(df[['SYMBOL','TimeStamp','Close','ema9','ema9_angle','linear_reg_slope','K_slow','D_slow','ST_Upperband','ST_Lowerband']].tail(5))

                    print (datetime.now().strftime('%H:%M:%S'))
                    limitprice = round((bidPrice-0.01),2)
                    print('Sell Stock for ', symbol, 'at', limitprice, 'with p/l = $', openpl)

                    #if (openpl > triggerprice):
                        #print('openpl > triggerprice = True')
                    #if (df['ST_Upperband'][previous_key]> 0) :
                        #print('Supertrend Line changed color = True')


                    profitloss = round((quantity * (limitprice-avgPrice)),2)


                    #sellEQUITY in TS
                    sellEQUITY(symbol, quantity, limitprice, tradelog_filename)

                    #add to DB
                    addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H-%M-%S'), str(symbol), 'SELL_STOCK', limitprice, quantity)
                    addprofitlossOutputtoDB(profitloss)
                    deletePositionfromDB(symbol)

                    #add log to DB
                    addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELL_STOCK', limitprice, quantity, profitloss, bidPrice, askPrice,df)


            if (positionType == 'Short'):        


                #if (openpl_percent >= 0.04) :
                #if (openpl_percent >= 0.02) or (openpl_percent < -0.60) :                
                #((df['K_slow'][previous_key] > df['D_slow'][previous_key]) and \
                 #(df['K_slow'][previous_key] > 30)) :
                #if ((openpl > triggerprice) and (df['HA_Close'][previous_key] > df['HA_Open'][previous_key])) or \
                    #(askPrice >= stop_loss):

                #if (openpl_percent >= 1.00) or (askPrice >= stop_loss) :  
                
                #if  (df['ST_Lowerband'][previous_key]> 0) or (df['ST_Lowerband'][last_key]> 0):
                #if (openpl >= triggerprice) or (askPrice >= stop_loss) :
                #if  (df['ST_Lowerband'][last_key] > 0) : 
                
                if (df['ST_Lowerband'][previous_key]> 0) and (df['ST_Upperband'][previous_key2] > 0) :
                    
                #if (openpl_percent >= 0.1 or openpl_percent <= -0.30) :
                #if (openpl_percent >= 0.1) :

                    print(df[['SYMBOL','TimeStamp','Close','ema9','ema9_angle','linear_reg_slope','K_slow','D_slow','ST_Upperband','ST_Lowerband']].tail(5))

                    print (datetime.now().strftime('%H:%M:%S'))
                    limitprice = round((askPrice+0.01),2)

                    print('Buy to Cover for ', symbol, 'at', limitprice, 'with p/l = $', openpl)

                    #if (openpl > triggerprice):
                        #print('openpl > triggerprice = True')
                    #if (df['ST_Lowerband'][previous_key]> 0) :
                        #print('Supertrend Line changed color = True')


                    profitloss = round((quantity * (avgPrice-limitprice)),2)

                    #buytocoverEQUITY in TS
                    buytocoverEQUITY(symbol, quantity, limitprice, tradelog_filename)

                    #add to DB
                    addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), str(symbol), 'BUYTOCOVER_STOCK', limitprice, quantity)
                    addprofitlossOutputtoDB(profitloss)
                    deletePositionfromDB(symbol)


                    #add log to DB
                    addOrderLogOutputtoDB(datetime_now, symbol, strategy,'BUYTOCOVER_STOCK', limitprice, quantity, profitloss, bidPrice, askPrice,df)

    #print("******************")
        
        
    except Exception as e:
        traceback.print_exc()
        print(e)
        pass


    
while True:
        
    now = datetime.now()
    today801=now.replace(hour=8,minute=0,second=0,microsecond=0)
    today1500=now.replace(hour=15,minute=0,second=0,microsecond=0)

    if (now > today801 and now < today1500 ) :
        #symbol_list = getStocks()
        #print(symbol_list)
        #getStrategyResult(symbol_list)
        
        #Set trade signals to True or False based on market movement
        sellstrategy = get_Sell_Strategy_Trade_Signals_fromDB()
        if (sellstrategy == True):
            
            # Time starts from here
            start = time.time()       

            #update option values in the DB - moved to separate
            #updatePositionValuesinDB()    

            positions = getDFFromPositionTable()
            #print(positions)


            if (len(positions) > 0):
                with concurrent.futures.ThreadPoolExecutor(max_workers=len(positions)) as executor:
                    #print(threading.currentThread().ident)            
                    executor.map(getStrategyResult,positions)


            # Total time calculated
            print(now, ' - The program takes ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(10)

        '''
        # close all positions on market close or certain time    
        marketclose=now.replace(hour=14,minute=59,second=0,microsecond=0)
        if (now > marketclose) :
            positions = getDFFromPositionTable()
            if (len(positions) > 0):
                with concurrent.futures.ThreadPoolExecutor(max_workers=len(positions)) as executor:
                    #print(threading.currentThread().ident)            
                    executor.map(closeAllPositionsBeforeMarketClose,positions)
        break       

        '''

    
        

